### Imports and Setup

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from utils.models import model_mapping
import pandas as pd
import numpy as np
import json
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#runs = 10 # for ease when coding
runs = 100 # for final experiments
np.random.seed(42)
random_seeds = np.random.randint(0, 10000, size=runs)

dataset_names = ['rarefied', 'clr']
data_dir = "./data"
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

final_results = {}

### Looping Over Datasets

In [2]:
for dataset_name in dataset_names:
    print(f"\n🔍 Running Training Pipeline on: {dataset_name}")

    # loading data
    df = pd.read_csv(f"{data_dir}/{dataset_name}.csv")
    X = df.drop(columns=["Wine"])
    y = df["Wine"]

    # encoding labels
    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # initializing models from mapping
    models = {name: model_class() for name, model_class in model_mapping.items()}
    # dictionary to store model evaluation results
    model_results = {name: np.zeros((runs, 5)) for name in models}

    # Training Loop
    for i, seed in enumerate(random_seeds):
        # showing progress
        if (i + 1) % 10 == 0 or i == 0:
            print(f"Run {i + 1}/{runs} (seed={seed})")
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
        
        # preprocessing
        vt = VarianceThreshold(threshold=0)
        X_train = vt.fit_transform(X_train)
        X_test = vt.transform(X_test)
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # training & evaluating each model
        for name, current_model in models.items():
            current_model.train(X_train_scaled, y_train)
            model_results[name][i, :] = current_model.predict(X_test_scaled, y_test)
            

    # computing average metrics
    average_results = {
    name: np.mean(results, axis=0) for name, results in model_results.items()
    }

    # best model selection with priority on AUC while using other metrics as tiebreakers
    best_model = max(average_results, key=lambda name: tuple(average_results[name]))
    best_metrics = average_results[best_model]

    print(f"\nBest Model for {dataset_name}: {best_model}")
    print(
        f"AUC: {best_metrics[0]:.5f}\n"
        f"Accuracy: {best_metrics[1]:.5f}\n"
        f"Precision: {best_metrics[2]:.5f}\n"
        f"Recall: {best_metrics[3]:.5f}\n"
        f"F1: {best_metrics[4]:.5f}\n"
    )

    # saving best model info in text format
    with open(f"{results_dir}/best_model_{dataset_name}.txt", "w") as f:
        f.write(f"Best Model: {best_model}\n")
        f.write(
            f"AUC: {best_metrics[0]:.5f}\n"
            f"Accuracy: {best_metrics[1]:.5f}\n"
            f"Precision: {best_metrics[2]:.5f}\n"
            f"Recall: {best_metrics[3]:.5f}\n"
            f"F1: {best_metrics[4]:.5f}\n"
        )

    # storing results in dictionary for JSON
    final_results[dataset_name] = {
        "best_model": best_model,
        "metrics": {
            "auc": round(best_metrics[0], 5),
            "accuracy": round(best_metrics[1], 5),
            "precision": round(best_metrics[2], 5),
            "recall": round(best_metrics[3], 5),
            "f1": round(best_metrics[4], 5)
        }
    }

# saving structured info for frontend
with open(f"{results_dir}/best_models.json", "w") as f:
    json.dump(final_results, f, indent=2)


🔍 Running Training Pipeline on: rarefied
Run 1/100 (seed=7270)
Run 10/100 (seed=8322)
Run 20/100 (seed=3385)
Run 30/100 (seed=189)
Run 40/100 (seed=8838)
Run 50/100 (seed=7099)
Run 60/100 (seed=3843)
Run 70/100 (seed=1016)
Run 80/100 (seed=4859)
Run 90/100 (seed=5463)
Run 100/100 (seed=6184)

Best Model for rarefied: Linear Discriminant Analysis
AUC: 0.99982
Accuracy: 0.98611
Precision: 0.98725
Recall: 0.98611
F1: 0.98612


🔍 Running Training Pipeline on: clr
Run 1/100 (seed=7270)
Run 10/100 (seed=8322)
Run 20/100 (seed=3385)
Run 30/100 (seed=189)
Run 40/100 (seed=8838)
Run 50/100 (seed=7099)
Run 60/100 (seed=3843)
Run 70/100 (seed=1016)
Run 80/100 (seed=4859)
Run 90/100 (seed=5463)
Run 100/100 (seed=6184)

Best Model for clr: Linear Discriminant Analysis
AUC: 0.99982
Accuracy: 0.98611
Precision: 0.98725
Recall: 0.98611
F1: 0.98612

